In [1]:
#!apt-get update
!pip install xmltodict
!pip install pytz
!pip install requests

In [17]:
import time
import pandas as pd
import json
from datetime import datetime, timedelta, timezone
import xlrd
from collections import OrderedDict
import zipfile
import os
from bisect import bisect
import xmltodict
from ast import literal_eval
from bs4 import BeautifulSoup
from urllib import request
import requests
import csv

In [3]:
#설정
KST = timezone(timedelta(hours=9))
time_record = datetime.now(KST)
today = time_record.strftime('%Y%m%d')

start_date = datetime.now() + timedelta(days=-60)
end_date = datetime.now() + timedelta(days=-30)

excelfile_path = 'C:\\Users\\user\\Medicine\\download'
documentfile_path = 'C:\\Users\\user\\Medicine\\document'
dbfile_path = 'C:\\Users\\user\\Medicine\\db'
csvfile_path = 'C:\\Users\\user\\Medicine\\dbcsv'
chart_excelfile_path = 'C:\\Users\\user\\Medicine\\chart'
chart_db_csvfile_path = 'C:\\Users\\user\\Medicine\\chart_db'

#def_db = {'item_seq':'품목일련번호', 'item_name':'품목명', 'entp_name':'업체명', 'chart':'성상', 'item_permit_date':'허가일자', 'edi_code':'보험코드', 'permit_kind_name':'허가/신고구분', 'cancel_name':'취소상태', 'cancel_date':'취소일자', 'etc_otc_code':'전문일반', 'main_item_ingr':'주성분명', 'atc':'ATC코드'}
def_db = {'ITEM_SEQ':'품목일련번호', 'ITEM_NAME':'품목명', 'ENTP_NAME':'업체명', 'ITEM_PERMIT_DATE':'허가일자', 'ETC_OTC_CODE':'전문일반', 'CHART':'성상', 'BAR_CODE':'표준코드', 'MATERIAL_NAME':'원료성분', 'EE_DOC_ID':'효능효과', 'UD_DOC_ID':'용법용량', 'NB_DOC_ID':'주의사항', 'STORAGE_METHOD':'저장방법', 'VALID_TERM':'유효기간', 'REEXAM_TARGET':'재심사대상', 'REEXAM_DATE':'재심사기간', 'PACK_UNIT':'포장단위', 'EDI_CODE':'보험코드', 'PERMIT_KIND_NAME':'허가/신고구분', 'MAKE_MATERIAL_FLAG':'완제원료구분', 'NEWDRUG_CLASS_NAME':'신약여부', 'CANCEL_DATE':'취소일자', 'CANCEL_NAME':'취소상태', 'CHANGE_DATE': '변경일자', 'NARCOTIC_KIND_NAME':'마약류분류'}
def_chart = {'ITEM_SEQ':'품목일련번호', 'ITEM_NAME':'품목명', 'ENTP_SEQ':'업소일련번호', 'ENTP_NAME':'업소명', 'CHART':'성상', 'ITEM_IMAGE':'큰제품이미지', 'PRINT_FRONT':'표시앞', 'PRINT_BACK':'표시뒤', 'DRUG_SHAPE':'의약품제형', 'COLOR_CLASS1':'색상앞', 'COLOR_CLASS2':'색상뒤', 'LINE_FRONT':'분할선앞', 'LINE_BACK':'분할선뒤', 'LENG_LONG':'크기장축', 'LENG_SHORT':'크기단축', 'THICK':'크기두께', 'IMG_REGIST_TS':'이미지생성일자(약학정보원)', 'CLASS_NO':'분류번호', 'CLASS_NAME':'분류명', 'ETC_OTC_CODE':'전문일반구분', 'ITEM_PERMIT_DATE':'품목허가일자', 'FORM_CODE_NAME':'제형코드명', 'MARK_CODE_FRONT_ANAL': '표기내용앞', 'MARK_CODE_BACK_ANAL':'표기내용뒤', 'MARK_CODE_FRONT_IMG':'표기이미지앞', 'MARK_CODE_BACK_IMG':'표기이미지뒤', 'MARK_CODE_FRONT':'표기코드앞', 'MARK_CODE_BACK':'표기코드뒤', 'CHANGE_DATE':'변경일자'}
xmlfile_common_path = 'C:\\Users\\user\\Medicine\\unzip\\'
def_xmlfile_folder_path = [xmlfile_common_path+'1954-2000', xmlfile_common_path+'2001-2005', xmlfile_common_path+'2006-2010', xmlfile_common_path+'2011-2015', xmlfile_common_path+'2016-2020']

In [33]:
# 최초 전체 생성
#download_excelfile_by_api(today, excelfile_path)
df = load_file(today, excelfile_path)
out_json = get_list_from_db('save', df)
save_or_update(out_json, 'db', dbfile_path)

get_list_from_db end & make_db_json_list start
2022-06-20 13:55:21.376040+09:00
make_db_json_list end & get_list_from_db end
2022-06-20 13:55:37.168846+09:00
1955_00001-00100
1956_00001-00100
1957_00001-00100
1958_00001-00100
1959_00001-00100
1960_00001-00100
1961_00001-00100
1962_00001-00100
1963_00001-00100
1964_00001-00100
1965_00001-00100
1966_00001-00100
1967_00001-00100
1968_00001-00100
1969_00001-00100
1969_00101-00200
1970_00001-00100
1970_00101-00200
1970_00201-00300
1971_00001-00100
1971_00101-00200
1971_00201-00300
1972_00001-00100
1972_00101-00200
1972_00201-00300
1972_00301-00400
1972_00401-00500
1973_00001-00100
1973_00101-00200
1973_00201-00300
1974_00001-00100
1974_00101-00200
1974_00201-00300
1974_00301-00400
1974_00401-00500
1974_00501-00600
1975_00001-00100
1975_00101-00200
1975_00201-00300
1975_00301-00400
1975_00401-00500
1975_00501-00600
1975_00601-00700
1976_00001-00100
1976_00101-00200
1976_00301-00400
1976_00401-00500
1976_00501-00600
1976_00601-00700
1976_0070

1996_04901-05000
1996_05101-05200
1997_00001-00100
1997_00101-00200
1997_00201-00300
1997_00301-00400
1997_00401-00500
1997_00501-00600
1997_00601-00700
1997_00701-00800
1997_00801-00900
1997_00901-01000
1997_01001-01100
1997_01101-01200
1997_01201-01300
1997_01301-01400
1997_01401-01500
1997_01501-01600
1997_01601-01700
1997_01701-01800
1997_01801-01900
1997_01901-02000
1997_02001-02100
1997_02101-02200
1997_02201-02300
1997_02301-02400
1997_02401-02500
1997_02501-02600
1997_02601-02700
1997_02701-02800
1997_02801-02900
1997_02901-03000
1997_03001-03100
1997_03101-03200
1997_03201-03300
1997_03301-03400
1997_03401-03500
1997_03501-03600
1997_03601-03700
1997_04201-04300
1997_04301-04400
1997_04401-04500
1997_04501-04600
1997_04601-04700
1997_04701-04800
1998_00001-00100
1998_00101-00200
1998_00201-00300
1998_00301-00400
1998_00401-00500
1998_00501-00600
1998_00601-00700
1998_00701-00800
1998_00801-00900
1998_00901-01000
1998_01001-01100
1998_01101-01200
1998_01201-01300
1998_01301-014

2005_00801-00900
2005_00901-01000
2005_01001-01100
2005_01101-01200
2005_01201-01300
2005_01301-01400
2005_01401-01500
2005_01501-01600
2005_01601-01700
2005_01701-01800
2005_01801-01900
2005_01901-02000
2005_02001-02100
2005_02101-02200
2005_02201-02300
2005_02301-02400
2005_02401-02500
2005_02501-02600
2005_02601-02700
2005_02701-02800
2005_02801-02900
2005_02901-03000
2005_03001-03100
2005_03101-03200
2005_03201-03300
2005_03301-03400
2005_03401-03500
2005_03501-03600
2005_03601-03700
2005_03701-03800
2005_03801-03900
2005_03901-04000
2005_05501-05600
2005_05601-05700
2005_05701-05800
2005_05901-06000
2005_06201-06300
2005_10901-11000
2005_11001-11100
2005_11101-11200
2005_11201-11300
2005_11301-11400
2005_11401-11500
2005_11601-11700
2005_11701-11800
2005_11801-11900
2005_11901-12000
2005_12001-12100
2005_12101-12200
2005_12201-12300
2005_12301-12400
2006_00001-00100
2006_00101-00200
2006_00201-00300
2006_00301-00400
2006_00401-00500
2006_00501-00600
2006_00601-00700
2006_00701-008

2010_01401-01500
2010_01501-01600
2010_01601-01700
2010_01701-01800
2010_01801-01900
2010_01901-02000
2010_02001-02100
2010_02101-02200
2010_02201-02300
2010_02301-02400
2010_02401-02500
2010_02501-02600
2010_02601-02700
2010_02701-02800
2010_02801-02900
2010_02901-03000
2010_03001-03100
2010_03101-03200
2010_03201-03300
2010_03301-03400
2010_03401-03500
2010_03501-03600
2010_03601-03700
2010_03701-03800
2010_03801-03900
2010_03901-04000
2010_04001-04100
2010_04101-04200
2010_04201-04300
2010_04301-04400
2010_04401-04500
2010_04501-04600
2010_04601-04700
2010_04701-04800
2010_04801-04900
2010_04901-05000
2010_05001-05100
2010_05101-05200
2010_05201-05300
2010_05401-05500
2010_05501-05600
2010_05601-05700
2010_05701-05800
2010_05801-05900
2010_05901-06000
2010_06001-06100
2010_06101-06200
2010_06201-06300
2010_06301-06400
2010_06401-06500
2010_06501-06600
2010_06601-06700
2010_06701-06800
2010_06801-06900
2010_06901-07000
2010_07001-07100
2010_07101-07200
2010_07201-07300
2010_07301-074

2015_05501-05600
2015_05601-05700
2015_05701-05800
2015_05801-05900
2015_05901-06000
2015_06001-06100
2015_06101-06200
2015_06201-06300
2015_06301-06400
2015_06401-06500
2015_06501-06600
2015_06601-06700
2015_06701-06800
2015_06801-06900
2015_07001-07100
2015_07101-07200
2015_07201-07300
2015_07301-07400
2015_07401-07500
2015_07501-07600
2015_07601-07700
2015_07701-07800
2015_07801-07900
2015_07901-08000
2015_08001-08100
2015_08101-08200
2015_08201-08300
2015_08301-08400
2015_08401-08500
2015_08501-08600
2016_00001-00100
2016_00101-00200
2016_00201-00300
2016_00301-00400
2016_00401-00500
2016_00501-00600
2016_00601-00700
2016_00701-00800
2016_00801-00900
2016_00901-01000
2016_01001-01100
2016_01101-01200
2016_01201-01300
2016_01301-01400
2016_01401-01500
2016_01501-01600
2016_01601-01700
2016_01701-01800
2016_01801-01900
2016_01901-02000
2016_02001-02100
2016_02101-02200
2016_02201-02300
2016_02301-02400
2016_02401-02500
2016_02501-02600
2016_02601-02700
2016_02701-02800
2016_02801-029

2021_06901-07000
2021_07001-07100
2021_07101-07200
2021_07201-07300
2021_07301-07400
2021_07401-07500
2021_07501-07600
2021_07601-07700
2021_07701-07800
2021_07801-07900
2021_07901-08000
2021_08001-08100
2021_08101-08200
2021_08201-08300
2022_00001-00100
2022_00101-00200
2022_00201-00300
2022_00301-00400
2022_00401-00500
2022_00501-00600
2022_00601-00700
2022_00701-00800
2022_00801-00900
2022_00901-01000
2022_01001-01100
2022_01101-01200
2022_01201-01300
2022_01301-01400
2022_01401-01500
2022_01501-01600
2022_01601-01700
2022_01701-01800
2022_01801-01900
2022_01901-02000
2022_02001-02100
2022_02101-02200
2022_02201-02300
2022_02301-02400
2022_02401-02500
2022_02501-02600


In [13]:
#업데이트 - 추가
#download_excelfile_by_api(today, excelfile_path)
df = load_file(today, excelfile_path)
out_json = get_list_from_db('update', df, start_date, end_date, '허가일자')
save_or_update(out_json, 'db', dbfile_path)

#새로운 허가분에 대한 item_seq의 document 저장
missing_seq_list = get_item_seq_list_from_out_json(out_json);
_out_json = get_list_from_api(missing_seq_list)
save_or_update(_out_json, 'document', documentfile_path)

get_list_from_db end & make_db_json_list start
2022-07-08 10:34:49.756519+09:00
make_db_json_list end & get_list_from_db end
2022-07-08 10:34:49.836303+09:00
202201862
202201863
202201864
202201869
202201872
202201873
202201874
202201875
202201876
202201877
202201878
202201879
202201880
202201881
202201882
202201883
202201884
202201885
202201886
202201888
202201890
202201893
202201894
202201895
202201898
202201899
202201900
202201902
202201903
202201915
202201921
202201922
202201930
202201931
202201932
202201933
202201934
202201935
202201936
202201937
202201938
202201939
202201945
202201946
202201947
202201952
202201953
202201954
202201955
202201956
202201957
202201958
202201959
202201960
202201961
202201962
202201963
202201964
202201965
202201966
202201967
202201968
202201969
202201970
202201982
202201983
202201984
202201988
202202000
202202001
202202002
202202003
202202007
202202008
202202009
202202010
202202011
202202012
202202013
202202014
202202015
202202016
202202017
202202018
20

In [14]:
#업데이트 - 변경일자
#download_excelfile_by_api(today, excelfile_path)
df = load_file(today, excelfile_path)
out_json = get_list_from_db('update', df, start_date, end_date, '변경일자')
save_or_update(out_json, 'db', dbfile_path)

get_list_from_db end & make_db_json_list start
2022-07-08 10:42:55.480139+09:00
make_db_json_list end & get_list_from_db end
2022-07-08 10:42:55.669623+09:00
1970_00201-00300
1971_00201-00300
1984_01301-01400
1986_01001-01100
1986_01201-01300
1988_02601-02700
1990_00501-00600
1990_01301-01400
1990_02301-02400
1991_01401-01500
1991_02701-02800
1992_00101-00200
1992_01901-02000
1992_02501-02600
1992_02901-03000
1993_01401-01500
1993_01901-02000
1993_02601-02700
1994_00301-00400
1994_01401-01500
1994_01501-01600
1994_02301-02400
1994_03201-03300
1995_00001-00100
1996_01001-01100
1996_02701-02800
1997_00801-00900
1997_01301-01400
1997_01401-01500
1997_02201-02300
1997_02401-02500
1997_02901-03000
1997_03301-03400
1998_01501-01600
1998_03101-03200
1999_01501-01600
1999_02301-02400
1999_03201-03300
1999_03601-03700
1999_07201-07300
2000_01901-02000
2000_02301-02400
2000_03301-03400
2000_08901-09000
2000_09301-09400
2001_00101-00200
2001_00701-00800
2001_02601-02700
2002_00901-01000
2002_0270

In [15]:
#업데이트 - 취소일자
download_excelfile_by_api(today, excelfile_path)
df = load_file(today, excelfile_path)
out_json = get_list_from_db('update', df, start_date, end_date, '취소일자')
save_or_update(out_json, 'db', dbfile_path)

저장되었습니다.
get_list_from_db end & make_db_json_list start
2022-07-08 10:44:50.160081+09:00
make_db_json_list end & get_list_from_db end
2022-07-08 10:44:50.236825+09:00
1985_00901-01000
1986_00201-00300
1987_00901-01000
1988_01201-01300
1988_01301-01400
1989_01301-01400
1991_00401-00500
1992_01601-01700
1995_01301-01400
1995_01501-01600
1996_01501-01600
1997_01701-01800
1998_01601-01700
1999_01801-01900
2000_02001-02100
2001_01801-01900
2003_01301-01400
2003_07601-07700
2004_02001-02100
2004_09901-10000
2005_01701-01800
2005_02101-02200
2005_12101-12200
2006_00301-00400
2008_09701-09800
2008_10701-10800
2009_00201-00300
2011_03401-03500
2016_02001-02100
2017_02301-02400
2017_02401-02500
2017_02501-02600
2017_02601-02700
2017_02701-02800
2017_03001-03100
2017_03101-03200
2017_03901-04000
2017_05601-05700
2017_07001-07100
2017_08601-08700
2018_02701-02800
2019_04501-04600
2020_01801-01900
2020_01901-02000
2020_04001-04100
2020_05001-05100
2020_08001-08100
2021_03301-03400
2021_06801-06900


In [4]:
#document 생성
for path in def_xmlfile_folder_path:
    time_record_document_start = datetime.now(KST)
    print('get_list_from_document start')
    print(time_record_document_start)
    out_json = get_list_from_document(path)
    time_record_document_end = datetime.now(KST)
    print('get_list_from_document end & save start')
    print(time_record_document_end)
    save_or_update(out_json, 'document', documentfile_path)
    time_record_save_end = datetime.now(KST)
    print('save end')
    print(time_record_save_end)

get_list_from_document start
2022-06-21 14:40:17.504414+09:00
201600079
201600080
201600081
201600103
201600113
201600129
201600166
201600170
201600171
201600172
201600173
201600178
201600179
201600185
201600186
201600189
201600190
201600191
201600196
201600204
201600206
201600207
201600208
201600209
201600210
201600211
201600217
201600218
201600243
201600249
201600250
201600251
201600253
201600254
201600259
201600260
201600261
201600269
201600274
201600275
201600278
201600289
201600290
201600297
201600298
201600299
201600300
201600301
201600302
201600303
201600304
201600315
201600316
201600317
201600318
201600319
201600320
201600329
201600332
201600339
201600340
201600356
201600357
201600358
201600359
201600361
201600362
201600365
201600366
201600370
201600371
201600378
201600379
201600380
201600385
201600386
201600387
201600388
201600389
201600390
201600391
201600392
201600396
201600397
201600407
201600419
201600420
201600425
201600426
201600427
201600428
201600429
201600430
20160043

201602584
201602588
201602590
201602591
201602592
201602600
201602601
201602602
201602608
201602610
201602611
201602615
201602616
201602619
201602624
201602629
201602637
201602638
201602639
201602640
201602641
201602642
201602651
201602652
201602656
201602657
201602659
201602662
201602663
201602668
201602669
201602670
201602671
201602676
201602677
201602678
201602679
201602680
201602682
201602683
201602684
201602688
201602693
201602700
201602701
201602702
201602703
201602704
201602705
201602706
201602707
201602708
201602709
201602713
201602714
201602722
201602723
201602724
201602732
201602733
201602734
201602735
201602736
201602737
201602738
201602739
201602740
201602741
201602760
201602761
201602762
201602763
201602764
201602765
201602766
201602768
201602769
201602770
201602771
201602772
201602773
201602774
201602775
201602776
201602777
201602778
201602779
201602780
201602781
201602784
201602785
201602786
201602789
201602795
201602796
201602797
201602798
201602799
201602800
201602801


201603923
201603924
201603925
201603926
201603927
201603928
201603929
201603931
201603932
201603933
201603934
201603935
201603936
201603937
201603938
201603939
201603940
201603941
201603942
201603943
201603944
201603945
201603947
201603948
201603949
201603950
201603951
201603952
201603953
201603954
201603955
201603956
201603957
201603958
201603959
201603960
201603961
201603962
201603963
201603964
201603965
201603966
201603967
201603968
201603969
201603970
201603971
201603972
201603973
201603974
201603975
201603984
201603985
201603986
201603987
201603988
201603989
201603990
201603992
201603993
201603994
201603995
201603996
201603997
201603998
201603999
201604000
201604001
201604002
201604003
201604005
201604006
201604007
201604008
201604009
201604010
201604011
201604012
201604013
201604014
201604015
201604017
201604018
201604019
201604020
201604021
201604022
201604023
201604024
201604030
201604031
201604032
201604033
201604034
201604035
201604036
201604037
201604044
201604045
201604046


201605993
201605995
201606000
201606003
201606005
201606006
201606012
201606019
201606029
201606030
201606031
201606040
201606041
201606042
201606043
201606044
201606045
201606046
201606047
201606048
201606049
201606050
201606051
201606052
201606053
201606054
201606055
201606056
201606057
201606058
201606059
201606067
201606068
201606071
201606074
201606075
201606080
201606081
201606082
201606083
201606084
201606085
201606123
201606139
201606147
201606148
201606149
201606162
201606167
201606187
201606189
201606194
201606196
201606201
201606202
201606204
201606205
201606213
201606230
201606231
201606240
201606241
201606242
201606244
201606245
201606252
201606253
201606254
201606255
201606256
201606257
201606258
201606259
201606261
201606263
201606264
201606273
201606276
201606292
201606293
201606294
201606295
201606297
201606306
201606307
201606313
201606314
201606315
201606316
201606317
201606318
201606321
201606322
201606327
201606328
201606331
201606332
201606333
201606334
201606336


201701479
201701480
201701481
201701482
201701483
201701484
201701498
201701499
201701500
201701501
201701502
201701503
201701505
201701506
201701508
201701509
201701511
201701512
201701513
201701519
201701521
201701522
201701523
201701527
201701528
201701529
201701530
201701533
201701536
201701538
201701540
201701541
201701544
201701545
201701546
201701547
201701548
201701549
201701551
201701552
201701553
201701554
201701555
201701556
201701557
201701558
201701559
201701563
201701564
201701565
201701568
201701569
201701570
201701571
201701572
201701574
201701576
201701586
201701587
201701601
201701602
201701611
201701612
201701613
201701617
201701619
201701634
201701635
201701674
201701678
201701679
201701682
201701717
201701718
201701734
201701735
201701736
201701737
201701738
201701748
201701749
201701750
201701751
201701752
201701753
201701755
201701756
201701774
201701800
201701825
201701828
201701831
201701832
201701833
201701834
201701835
201701836
201701837
201701838
201701839


...

201903396
201903397
201903398
201903399
201903400
201903402
201903403
201903405
201903406
201903407
201903408
201903409
201903410
201903412
201903415
201903416
201903417
201903418
201903419
201903420
201903421
201903422
201903423
201903424
201903425
201903426
201903427
201903428
201903431
201903432
201903433
201903434
201903436
201903437
201903438
201903439
201903440
201903442
201903443
201903444
201903445
201903446
201903448
201903450
201903451
201903452
201903453
201903455
201903456
201903457
201903458
201903459
201903460
201903461
201903462
201903463
201903465
201903466
201903471
201903472
201903473
201903474
201903475
201903476
201903477
201903478
201903479
201903480
201903481
201903482
201903483
201903484
201903485
201903486
201903487
201903488
201903489
201903490
201903491
201903492
201903493
201903494
201903495
201903496
201903497
201903498
201903499
201903500
201903501
201903502
201903503
201903504
201903505
201903506
201903507
201903508
201903509
201903510
201903511
201903515


201905101
201905234
201905235
201905236
201905237
201905238
201905239
201905240
201905241
201905242
201905243
201905244
201905257
201905258
201905261
201905262
201905263
201905264
201905265
201905266
201905267
201905268
201905269
201905270
201905273
201905276
201905277
201905278
201905279
201905280
201905282
201905283
201905284
201905285
201905286
201905287
201905288
201905290
201905292
201905293
201905296
201905297
201905298
201905299
201905300
201905301
201905303
201905304
201905305
201905306
201905307
201905308
201905309
201905310
201905311
201905312
201905313
201905314
201905315
201905316
201905317
201905318
201905321
201905322
201905326
201905327
201905329
201905330
201905331
201905338
201905339
201905340
201905341
201905342
201905343
201905344
201905346
201905347
201905348
201905349
201905350
201905352
201905353
201905355
201905356
201905357
201905358
201905359
201905360
201905362
201905363
201905364
201905365
201905366
201905367
201905368
201905369
201905370
201905371
201905372


201906565
201906566
201906567
201906568
201906569
201906571
201906572
201906573
201906574
201906575
201906576
201906577
201906578
201906579
201906580
201906581
201906582
201906583
201906584
201906585
201906586
201906587
201906588
201906589
201906590
201906596
201906602
201906603
201906604
201906607
201906608
201906609
201906610
201906612
201906616
201906617
201906623
201906624
201906625
201906626
201906627
201906628
201906629
201906632
201906633
201906634
201906636
201906637
201906638
201906639
201906643
201906644
201906645
201906646
201906647
201906648
201906649
201906667
201906668
201906669
201906670
201906672
201906673
201906674
201906675
201906677
201906678
201906679
201906680
201906681
201906686
201906687
201906688
201906689
201906691
201906692
201906693
201906694
201906695
201906696
201906697
201906742
201906751
201906752
201906754
201906755
201906768
201906769
201906770
201906789
201906790
201906791
201906797
201906798
201906800
201906817
201906818
201906822
201906823
201906824


201907917
201907920
201907921
201907923
201907924
201907925
201907926
201907927
201907928
201907929
201907930
201907931
201907932
201907933
201907936
201907937
201907938
201907939
201907940
201907941
201907942
201907947
201907948
201907949
201907950
201907953
201907955
201907956
201907957
201907958
201907959
201907960
201907961
201907962
201907963
201907965
201907967
201907968
201907969
201907970
201907971
201907980
201907981
201907982
201907985
201907986
201907987
201907988
201907989
201907992
201907993
201907995
201907996
201907997
201907999
201908000
201908001
201908002
201908003
201908004
201908005
201908006
201908007
201908008
201908009
201908010
201908011
201908012
201908013
201908014
201908015
201908017
201908018
201908019
201908022
201908023
201908025
201908026
201908027
201908028
201908029
201908030
201908031
201908032
201908036
201908037
201908038
201908039
201908040
201908041
201908042
201908043
201908047
201908049
201908050
201908051
201908052
201908054
201908055
201908057


2019_03401-03500
2019_03501-03600
2019_03601-03700
2019_03701-03800
2019_03801-03900
2019_04101-04200
2019_04201-04300
2019_04301-04400
2019_04401-04500
2019_04501-04600
2019_04601-04700
2019_04701-04800
2019_04801-04900
2019_05001-05100
2019_05101-05200
2019_05201-05300
2019_05301-05400
2019_05401-05500
2019_05501-05600
2019_05601-05700
2019_05701-05800
2019_05801-05900
2019_05901-06000
2019_06001-06100
2019_06101-06200
2019_06201-06300
2019_06301-06400
2019_06401-06500
2019_06501-06600
2019_06601-06700
2019_06701-06800
2019_06801-06900
2019_06901-07000
2019_07001-07100
2019_07101-07200
2019_07201-07300
2019_07301-07400
2019_07401-07500
2019_07501-07600
2019_07601-07700
2019_07701-07800
2019_07801-07900
2019_07901-08000
2019_08001-08100
2019_08101-08200
2019_08201-08300
2019_08301-08400
2019_08401-08500
2019_08501-08600
2019_08601-08700
save end
2022-06-21 14:48:33.035401+09:00


In [63]:
#엑셀파일과 비교했을 때 document.json에 누락된 item_seq 저장
df = load_file(today, excelfile_path)
missing_seq_list = get_missing_seq(df, documentfile_path)
print("missing_seq_list")
print(missing_seq_list)

#document api를 요청해서 document 생성
out_json = get_list_from_api(missing_seq_list)
save_or_update(out_json, 'document', documentfile_path)

195500002
195500005
195500006
195500014
195600004
195600006
195700004
195700007
195700008
195700009
195700010
195700013
195700014
195700015
195700016
195700020
195700033
195800018
195800019
195800020
195900009
195900034
195900043
195900045
195900051
196000001
196000002
196000008
196000010
196000011
196100001
196100002
196100007
196100008
196100042
196200032
196200034
196200043
196300001
196300011
196300025
196300034
196300035
196300043
196300044
196300064
196300065
196300078
196300084
196400020
196400036
196400037
196400046
196400054
196400082
196400089
196400090
196400099
196500004
196500034
196500051
196600011
196600012
196600018
196600062
196600063
196700013
196700014
196700015
196700060
196700061
196800016
196800017
196800019
196800036
196800053
196800062
196800088
196900021
196900027
196900028
196900031
196900033
196900042
196900058
196900068
196900081
196900082
196900101
196900130
196900131
197000005
197000013
197000014
197000015
197000016
197000017
197000026
197000037
197000040


198300024
198300026
198300028
198300029
198300030
198300032
198300033
198300036
198300038
198300041
198300042
198300043
198300044
198300045
198300055
198300064
198300065
198300066
198300067
198300073
198300091
198300092
198300096
198300097
198300098
198300104
198300113
198300114
198300115
198300121
198300131
198300132
198300133
198300142
198300145
198300146
198300147
198300150
198300171
198300174
198300175
198300176
198300216
198300217
198300219
198300222
198300223
198300224
198300225
198300226
198300227
198300230
198300232
198300234
198300235
198300236
198300237
198300238
198300240
198300241
198300242
198300243
198300244
198300246
198300247
198300251
198300253
198300254
198300256
198300257
198300261
198300266
198300268
198300269
198300271
198300273
198300274
198300276
198300277
198300278
198300279
198300280
198300281
198300282
198300287
198300290
198300292
198300295
198300315
198300327
198300328
198300329
198300342
198300343
198300370
198300374
198300389
198300393
198300402
198300420


198701888
198701947
198701948
198701949
198701954
198702190
198702202
198702203
198702205
198702208
198702209
198702210
198702211
198800027
198800042
198800043
198800044
198800057
198800062
198800063
198800064
198800065
198800066
198800067
198800068
198800069
198800070
198800071
198800072
198800073
198800074
198800075
198800076
198800077
198800078
198800079
198800080
198800081
198800082
198800083
198800084
198800085
198800086
198800087
198800088
198800089
198800090
198800091
198800092
198800093
198800094
198800095
198800096
198800097
198800098
198800099
198800100
198800101
198800102
198800103
198800104
198800105
198800106
198800107
198800108
198800109
198800110
198800111
198800112
198800113
198800114
198800115
198800116
198800118
198800119
198800127
198800128
198800137
198800140
198800141
198800142
198800143
198800150
198800153
198800154
198800155
198800167
198800168
198800169
198800170
198800175
198800186
198800190
198800191
198800192
198800193
198800220
198800230
198800231
198800234


199102608
199102609
199102630
199102670
199102690
199102713
199102714
199102732
199102737
199102741
199102767
199102769
199102787
199102788
199102792
199102825
199102830
199102861
199102938
199102948
199102954
199102956
199102966
199103258
199103287
199103295
199103325
199200067
199200079
199200081
199200082
199200083
199200084
199200085
199200091
199200094
199200096
199200097
199200099
199200100
199200103
199200104
199200106
199200108
199200144
199200163
199200164
199200171
199200173
199200174
199200179
199200198
199200199
199200201
199200202
199200203
199200204
199200205
199200210
199200211
199200212
199200219
199200220
199200221
199200224
199200225
199200227
199200228
199200229
199200230
199200231
199200232
199200234
199200235
199200236
199200248
199200249
199200258
199200261
199200268
199200269
199200271
199200286
199200305
199200306
199200308
199200312
199200313
199200319
199200324
199200342
199200349
199200350
199200351
199200352
199200354
199200361
199200406
199200407
199200412


199500295
199500300
199500302
199500305
199500306
199500307
199500311
199500313
199500317
199500319
199500320
199500322
199500327
199500329
199500330
199500331
199500334
199500335
199500348
199500352
199500368
199500372
199500386
199500388
199500389
199500390
199500395
199500396
199500397
199500400
199500405
199500413
199500419
199500420
199500428
199500435
199500439
199500441
199500442
199500452
199500463
199500466
199500470
199500476
199500481
199500490
199500494
199500495
199500497
199500498
199500501
199500503
199500505
199500507
199500508
199500513
199500530
199500536
199500549
199500573
199500575
199500578
199500628
199500630
199500642
199500646
199500647
199500656
199500666
199500674
199500687
199500692
199500697
199500707
199500734
199500752
199500791
199500795
199500807
199500818
199500819
199500820
199500823
199500827
199500842
199500845
199500849
199500850
199500892
199500895
199500901
199500902
199500908
199500958
199500959
199500967
199501023
199501025
199501026
199501037


...

202003968
202003969
202003970
202003971
202003972
202003973
202003974
202003975
202003976
202003978
202003979
202003980
202003981
202003983
202003984
202003987
202003988
202003989
202003991
202003992
202003994
202003995
202003996
202003997
202003998
202003999
202004000
202004001
202004002
202004003
202004004
202004005
202004006
202004007
202004008
202004009
202004010
202004011
202004012
202004013
202004014
202004015
202004017
202004018
202004020
202004021
202004022
202004023
202004024
202004025
202004026
202004027
202004028
202004029
202004030
202004031
202004032
202004033
202004034
202004035
202004036
202004037
202004038
202004039
202004040
202004041
202004042
202004043
202004044
202004045
202004046
202004047
202004048
202004049
202004050
202004051
202004052
202004053
202004054
202004055
202004056
202004057
202004058
202004059
202004060
202004061
202004062
202004063
202004064
202004065
202004066
202004067
202004068
202004069
202004076
202004077
202004078
202004079
202004081
202004082


202009439
202009440
202009441
202009443
202009444
202009445
202100007
202100011
202100014
202100015
202100019
202100024
202100035
202100037
202100038
202100039
202100040
202100041
202100043
202100044
202100045
202100046
202100047
202100051
202100053
202100065
202100066
202100068
202100076
202100077
202100078
202100082
202100083
202100084
202100085
202100086
202100087
202100088
202100089
202100090
202100091
202100092
202100093
202100097
202100098
202100099
202100100
202100101
202100102
202100127
202100128
202100129
202100130
202100138
202100139
202100140
202100141
202100142
202100143
202100146
202100147
202100148
202100149
202100150
202100162
202100172
202100174
202100175
202100176
202100177
202100178
202100179
202100180
202100181
202100182
202100183
202100184
202100185
202100186
202100187
202100188
202100189
202100190
202100191
202100192
202100193
202100194
202100195
202100196
202100197
202100198
202100199
202100200
202100201
202100202
202100203
202100204
202100205
202100206
202100207


202105472
202105480
202105483
202105485
202105492
202105501
202105502
202105509
202105531
202105532
202105533
202105534
202105535
202105536
202105537
202105545
202105546
202105573
202105574
202105575
202105576
202105577
202105578
202105579
202105581
202105592
202105596
202105597
202105598
202105599
202105607
202105608
202105609
202105610
202105611
202105612
202105613
202105614
202105615
202105616
202105617
202105619
202105625
202105645
202105647
202105656
202105657
202105658
202105666
202105680
202105691
202105692
202105693
202105711
202105715
202105716
202105717
202105718
202105719
202105720
202105725
202105726
202105730
202105731
202105737
202105738
202105747
202105752
202105753
202105763
202105769
202105771
202105782
202105792
202105795
202105807
202105831
202105840
202105841
202105842
202105844
202105848
202105849
202105901
202105907
202105908
202105922
202105927
202105928
202105942
202105952
202105957
202105959
202105960
202105978
202105979
202105987
202105999
202106002
202106007


202201863
202201864
202201869
202201872
202201873
202201874
202201875
202201876
202201877
202201878
202201879
202201880
202201881
202201882
202201883
202201884
202201885
202201886
202201888
202201890
202201893
202201894
202201895
202201898
202201899
202201900
202201902
202201903
202201915
202201921
202201922
202201930
202201931
202201932
202201933
202201934
202201935
202201936
202201937
202201938
202201939
202201945
202201946
202201947
202201952
202201953
202201954
202201955
202201956
202201957
202201958
202201959
202201960
202201961
202201962
202201963
202201964
202201965
202201966
202201967
202201968
202201969
202201970
202201982
202201983
202201984
202201988
202202000
202202001
202202002
202202003
202202007
202202008
202202009
202202010
202202011
202202012
202202013
202202014
202202015
202202016
202202017
202202018
202202019
202202020
202202021
202202022
202202023
202202024
202202025
202202026
202202027
202202028
202202029
202202031
202202036
202202039
202202040
202202041
202202061


KeyboardInterrupt: 

In [16]:
#엑셀파일과 비교했을 때 document.json에 누락된 item_seq 저장
df = load_file(today, excelfile_path)
time_record_missing_seq_start = datetime.now(KST)
print('get_missing_seq start')
print(time_record_missing_seq_start)
missing_seq_list = get_missing_seq(df, documentfile_path)
print(missing_seq_list)
time_record_missing_seq_end = datetime.now(KST)
print('get_missing_seq end')
print(time_record_missing_seq_end)

#document api를 요청해서 document 생성
time_record_api_seq_start = datetime.now(KST)
print('get_list_from_api start')
print(time_record_api_seq_start)
out_json = get_list_from_api(missing_seq_list)
time_record_api_seq_end = datetime.now(KST)
print('get_list_from_api end & save start')
print(time_record_api_seq_end)
save_or_update(out_json, 'document', documentfile_path)
time_record_save_end = datetime.now(KST)
print('save end')
print(time_record_save_end)

get_missing_seq start
2022-06-17 13:50:15.706570+09:00
document_1955_00001-00100.json
document_1956_00001-00100.json
document_1957_00001-00100.json
document_1958_00001-00100.json
document_1959_00001-00100.json
document_1960_00001-00100.json
document_1961_00001-00100.json
document_1962_00001-00100.json
document_1963_00001-00100.json
document_1964_00001-00100.json
document_1965_00001-00100.json
document_1966_00001-00100.json
document_1967_00001-00100.json
document_1968_00001-00100.json
document_1969_00001-00100.json
document_1969_00101-00200.json
document_1970_00001-00100.json
document_1970_00101-00200.json
document_1970_00201-00300.json
document_1971_00001-00100.json
document_1971_00101-00200.json
document_1971_00201-00300.json
document_1972_00001-00100.json
document_1972_00101-00200.json
document_1972_00201-00300.json
document_1972_00301-00400.json
document_1972_00401-00500.json
document_1973_00001-00100.json
document_1973_00101-00200.json
document_1973_00201-00300.json
document_1973_0

document_1991_00501-00600.json
document_1991_00601-00700.json
document_1991_00701-00800.json
document_1991_00801-00900.json
document_1991_00901-01000.json
document_1991_01001-01100.json
document_1991_01101-01200.json
document_1991_01201-01300.json
document_1991_01301-01400.json
document_1991_01401-01500.json
document_1991_01501-01600.json
document_1991_01601-01700.json
document_1991_01701-01800.json
document_1991_01801-01900.json
document_1991_01901-02000.json
document_1991_02001-02100.json
document_1991_02101-02200.json
document_1991_02201-02300.json
document_1991_02301-02400.json
document_1991_02401-02500.json
document_1991_02501-02600.json
document_1991_02601-02700.json
document_1991_02701-02800.json
document_1991_02801-02900.json
document_1991_02901-03000.json
document_1991_03201-03300.json
document_1991_03301-03400.json
document_1992_00001-00100.json
document_1992_00101-00200.json
document_1992_00201-00300.json
document_1992_00301-00400.json
document_1992_00401-00500.json
document

document_2002_02901-03000.json
document_2002_03001-03100.json
document_2002_03101-03200.json
document_2002_03201-03300.json
document_2002_03301-03400.json
document_2002_03401-03500.json
document_2002_03501-03600.json
document_2002_03601-03700.json
document_2002_03701-03800.json
document_2002_03901-04000.json
document_2002_04101-04200.json
document_2002_04201-04300.json
document_2002_04301-04400.json
document_2002_04401-04500.json
document_2002_05601-05700.json
document_2002_06401-06500.json
document_2002_06601-06700.json
document_2002_06801-06900.json
document_2002_06901-07000.json
document_2002_09301-09400.json
document_2002_09401-09500.json
document_2002_09601-09700.json
document_2002_09701-09800.json
document_2002_09801-09900.json
document_2002_09901-10000.json
document_2002_10001-10100.json
document_2002_10101-10200.json
document_2002_10201-10300.json
document_2002_10301-10400.json
document_2002_10501-10600.json
document_2002_10601-10700.json
document_2002_10701-10800.json
document

document_2006_11101-11200.json
document_2006_11301-11400.json
document_2006_11501-11600.json
document_2006_12001-12100.json
document_2006_12101-12200.json
document_2006_12201-12300.json
document_2006_12301-12400.json
document_2006_12401-12500.json
document_2006_12501-12600.json
document_2006_12601-12700.json
document_2006_12701-12800.json
document_2007_00001-00100.json
document_2007_00101-00200.json
document_2007_00201-00300.json
document_2007_00301-00400.json
document_2007_00401-00500.json
document_2007_00501-00600.json
document_2007_00601-00700.json
document_2007_00701-00800.json
document_2007_00801-00900.json
document_2007_00901-01000.json
document_2007_01001-01100.json
document_2007_01101-01200.json
document_2007_01201-01300.json
document_2007_01301-01400.json
document_2007_01401-01500.json
document_2007_01501-01600.json
document_2007_01601-01700.json
document_2007_01701-01800.json
document_2007_01801-01900.json
document_2007_01901-02000.json
document_2007_02001-02100.json
document

document_2009_00101-00200.json
document_2009_00201-00300.json
document_2009_00301-00400.json
document_2009_00401-00500.json
document_2009_00501-00600.json
document_2009_00601-00700.json
document_2009_00701-00800.json
document_2009_00801-00900.json
document_2009_00901-01000.json
document_2009_01001-01100.json
document_2009_01101-01200.json
document_2009_01201-01300.json
document_2009_01301-01400.json
document_2009_01401-01500.json
document_2009_01501-01600.json
document_2009_01601-01700.json
document_2009_01701-01800.json
document_2009_01801-01900.json
document_2009_01901-02000.json
document_2009_02001-02100.json
document_2009_02101-02200.json
document_2009_02201-02300.json
document_2009_02301-02400.json
document_2009_02401-02500.json
document_2009_02501-02600.json
document_2009_02601-02700.json
document_2009_02701-02800.json
document_2009_02801-02900.json
document_2009_02901-03000.json
document_2009_03001-03100.json
document_2009_03101-03200.json
document_2009_03201-03300.json
document

document_2011_10801-10900.json
document_2011_10901-11000.json
document_2011_11001-11100.json
document_2011_11101-11200.json
document_2011_11201-11300.json
document_2011_11401-11500.json
document_2011_11501-11600.json
document_2011_11801-11900.json
document_2011_12101-12200.json
document_2011_12201-12300.json
document_2011_12301-12400.json
document_2012_00001-00100.json
document_2012_00101-00200.json
document_2012_00201-00300.json
document_2012_00301-00400.json
document_2012_00401-00500.json
document_2012_00501-00600.json
document_2012_00601-00700.json
document_2012_00701-00800.json
document_2012_00801-00900.json
document_2012_00901-01000.json
document_2012_01001-01100.json
document_2012_01101-01200.json
document_2012_01201-01300.json
document_2012_01301-01400.json
document_2012_01401-01500.json
document_2012_01501-01600.json
document_2012_01601-01700.json
document_2012_01701-01800.json
document_2012_01801-01900.json
document_2012_01901-02000.json
document_2012_02001-02100.json
document

document_2014_06001-06100.json
document_2014_06101-06200.json
document_2014_06301-06400.json
document_2015_00001-00100.json
document_2015_00101-00200.json
document_2015_00201-00300.json
document_2015_00301-00400.json
document_2015_00401-00500.json
document_2015_00501-00600.json
document_2015_00601-00700.json
document_2015_00701-00800.json
document_2015_00801-00900.json
document_2015_00901-01000.json
document_2015_01001-01100.json
document_2015_01101-01200.json
document_2015_01201-01300.json
document_2015_01301-01400.json
document_2015_01401-01500.json
document_2015_01501-01600.json
document_2015_01601-01700.json
document_2015_01701-01800.json
document_2015_01801-01900.json
document_2015_01901-02000.json
document_2015_02001-02100.json
document_2015_02201-02300.json
document_2015_02301-02400.json
document_2015_02401-02500.json
document_2015_02501-02600.json
document_2015_02601-02700.json
document_2015_02701-02800.json
document_2015_02801-02900.json
document_2015_02901-03000.json
document

document_2018_03001-03100.json
document_2018_03101-03200.json
document_2018_03201-03300.json
document_2018_03301-03400.json
document_2018_03401-03500.json
document_2018_03501-03600.json
document_2018_03601-03700.json
document_2018_03701-03800.json
document_2018_03801-03900.json
document_2018_03901-04000.json
document_2018_04001-04100.json
document_2018_04101-04200.json
document_2018_04201-04300.json
document_2018_04301-04400.json
document_2018_04401-04500.json
document_2018_04501-04600.json
document_2018_04601-04700.json
document_2018_04701-04800.json
document_2018_04801-04900.json
document_2018_04901-05000.json
document_2018_05001-05100.json
document_2018_05101-05200.json
document_2018_05201-05300.json
document_2018_05301-05400.json
document_2018_05401-05500.json
document_2019_00001-00100.json
document_2019_00101-00200.json
document_2019_00201-00300.json
document_2019_00301-00400.json
document_2019_00501-00600.json
document_2019_00601-00700.json
document_2019_00701-00800.json
document

197400406
197600077
197700134
197800303
197800304
197800305
197800307
197800308
197800318
197800323
197800325
197800326
197800327
197800329
197800334
197800335
197800336
197800337
197800338
197800339
197800340
197800352
197800388
197800389
197800390
197800391
197900069
198000187
198000189
198000229
198000230
198100671
198100673
198100674
198300551
198301161
198400588
198400590
198500818
198501133
198501424
198600738


KeyboardInterrupt: 

In [18]:
#db.json을 csv로 바꿈
dbfile_name_list = os.listdir(dbfile_path)
check_first = True
for file_name in dbfile_name_list:
    print(file_name)
    cur_file_path = dbfile_path + '\\' + file_name
    cur_pdObj = json_to_pd(cur_file_path)
    cur_headers_list = cur_pdObj.columns.tolist()
    cur_values_list = cur_pdObj.values.tolist()
    write_csv(cur_headers_list, cur_values_list, check_first)
    check_first = False

db_1955_00001-00100.json
db_1956_00001-00100.json
db_1957_00001-00100.json
db_1958_00001-00100.json
db_1959_00001-00100.json
db_1960_00001-00100.json
db_1961_00001-00100.json
db_1962_00001-00100.json
db_1963_00001-00100.json
db_1964_00001-00100.json
db_1965_00001-00100.json
db_1966_00001-00100.json
db_1967_00001-00100.json
db_1968_00001-00100.json
db_1969_00001-00100.json
db_1969_00101-00200.json
db_1970_00001-00100.json
db_1970_00101-00200.json
db_1970_00201-00300.json
db_1971_00001-00100.json
db_1971_00101-00200.json
db_1971_00201-00300.json
db_1972_00001-00100.json
db_1972_00101-00200.json
db_1972_00201-00300.json
db_1972_00301-00400.json
db_1972_00401-00500.json
db_1973_00001-00100.json
db_1973_00101-00200.json
db_1973_00201-00300.json
db_1974_00001-00100.json
db_1974_00101-00200.json
db_1974_00201-00300.json
db_1974_00301-00400.json
db_1974_00401-00500.json
db_1974_00501-00600.json
db_1975_00001-00100.json
db_1975_00101-00200.json
db_1975_00201-00300.json
db_1975_00301-00400.json


db_1993_00501-00600.json
db_1993_00601-00700.json
db_1993_00701-00800.json
db_1993_00801-00900.json
db_1993_00901-01000.json
db_1993_01001-01100.json
db_1993_01101-01200.json
db_1993_01201-01300.json
db_1993_01301-01400.json
db_1993_01401-01500.json
db_1993_01501-01600.json
db_1993_01601-01700.json
db_1993_01701-01800.json
db_1993_01801-01900.json
db_1993_01901-02000.json
db_1993_02001-02100.json
db_1993_02101-02200.json
db_1993_02201-02300.json
db_1993_02301-02400.json
db_1993_02401-02500.json
db_1993_02501-02600.json
db_1993_02601-02700.json
db_1993_02701-02800.json
db_1993_02801-02900.json
db_1993_02901-03000.json
db_1993_03001-03100.json
db_1993_03101-03200.json
db_1993_03201-03300.json
db_1993_03901-04000.json
db_1993_04001-04100.json
db_1993_04101-04200.json
db_1993_04201-04300.json
db_1993_04301-04400.json
db_1993_04401-04500.json
db_1993_04501-04600.json
db_1994_00001-00100.json
db_1994_00101-00200.json
db_1994_00201-00300.json
db_1994_00301-00400.json
db_1994_00401-00500.json


db_2000_03901-04000.json
db_2000_04001-04100.json
db_2000_04101-04200.json
db_2000_04201-04300.json
db_2000_04301-04400.json
db_2000_04401-04500.json
db_2000_04501-04600.json
db_2000_04601-04700.json
db_2000_04801-04900.json
db_2000_04901-05000.json
db_2000_05001-05100.json
db_2000_05101-05200.json
db_2000_05201-05300.json
db_2000_05301-05400.json
db_2000_05401-05500.json
db_2000_05501-05600.json
db_2000_05601-05700.json
db_2000_05701-05800.json
db_2000_05801-05900.json
db_2000_05901-06000.json
db_2000_06301-06400.json
db_2000_06601-06700.json
db_2000_06701-06800.json
db_2000_06901-07000.json
db_2000_07001-07100.json
db_2000_08101-08200.json
db_2000_08501-08600.json
db_2000_08701-08800.json
db_2000_08801-08900.json
db_2000_08901-09000.json
db_2000_09001-09100.json
db_2000_09101-09200.json
db_2000_09201-09300.json
db_2000_09301-09400.json
db_2000_09401-09500.json
db_2000_09501-09600.json
db_2000_09601-09700.json
db_2000_09701-09800.json
db_2001_00001-00100.json
db_2001_00101-00200.json


db_2005_11601-11700.json
db_2005_11701-11800.json
db_2005_11801-11900.json
db_2005_11901-12000.json
db_2005_12001-12100.json
db_2005_12101-12200.json
db_2005_12201-12300.json
db_2005_12301-12400.json
db_2006_00001-00100.json
db_2006_00101-00200.json
db_2006_00201-00300.json
db_2006_00301-00400.json
db_2006_00401-00500.json
db_2006_00501-00600.json
db_2006_00601-00700.json
db_2006_00701-00800.json
db_2006_00801-00900.json
db_2006_00901-01000.json
db_2006_01001-01100.json
db_2006_01801-01900.json
db_2006_03801-03900.json
db_2006_03901-04000.json
db_2006_04001-04100.json
db_2006_04101-04200.json
db_2006_04201-04300.json
db_2006_04301-04400.json
db_2006_04401-04500.json
db_2006_04601-04700.json
db_2006_05201-05300.json
db_2006_05301-05400.json
db_2006_05401-05500.json
db_2006_05501-05600.json
db_2006_05601-05700.json
db_2006_06101-06200.json
db_2006_06201-06300.json
db_2006_06301-06400.json
db_2006_06401-06500.json
db_2006_06501-06600.json
db_2006_06601-06700.json
db_2006_06701-06800.json


db_2009_00601-00700.json
db_2009_00701-00800.json
db_2009_00801-00900.json
db_2009_00901-01000.json
db_2009_01001-01100.json
db_2009_01101-01200.json
db_2009_01201-01300.json
db_2009_01301-01400.json
db_2009_01401-01500.json
db_2009_01501-01600.json
db_2009_01601-01700.json
db_2009_01701-01800.json
db_2009_01801-01900.json
db_2009_01901-02000.json
db_2009_02001-02100.json
db_2009_02101-02200.json
db_2009_02201-02300.json
db_2009_02301-02400.json
db_2009_02401-02500.json
db_2009_02501-02600.json
db_2009_02601-02700.json
db_2009_02701-02800.json
db_2009_02801-02900.json
db_2009_02901-03000.json
db_2009_03001-03100.json
db_2009_03101-03200.json
db_2009_03201-03300.json
db_2009_03301-03400.json
db_2009_03401-03500.json
db_2009_03501-03600.json
db_2009_03601-03700.json
db_2009_03701-03800.json
db_2009_03801-03900.json
db_2009_03901-04000.json
db_2009_04001-04100.json
db_2009_04101-04200.json
db_2009_04201-04300.json
db_2009_04301-04400.json
db_2009_04401-04500.json
db_2009_04501-04600.json


db_2012_06501-06600.json
db_2012_06601-06700.json
db_2012_06701-06800.json
db_2012_06801-06900.json
db_2012_06901-07000.json
db_2012_07001-07100.json
db_2012_07101-07200.json
db_2012_07201-07300.json
db_2012_07301-07400.json
db_2012_07401-07500.json
db_2012_07501-07600.json
db_2012_07601-07700.json
db_2012_07701-07800.json
db_2012_08001-08100.json
db_2012_08101-08200.json
db_2012_08201-08300.json
db_2012_08401-08500.json
db_2012_08501-08600.json
db_2012_08601-08700.json
db_2012_08701-08800.json
db_2012_08801-08900.json
db_2012_08901-09000.json
db_2012_09001-09100.json
db_2012_09101-09200.json
db_2012_09201-09300.json
db_2012_09301-09400.json
db_2012_09401-09500.json
db_2012_09501-09600.json
db_2012_09601-09700.json
db_2012_09701-09800.json
db_2012_10101-10200.json
db_2012_10201-10300.json
db_2012_10501-10600.json
db_2012_10701-10800.json
db_2012_10801-10900.json
db_2012_11001-11100.json
db_2012_11101-11200.json
db_2012_11501-11600.json
db_2013_00001-00100.json
db_2013_00101-00200.json


db_2016_05301-05400.json
db_2016_05401-05500.json
db_2016_05501-05600.json
db_2016_05601-05700.json
db_2016_05701-05800.json
db_2016_05801-05900.json
db_2016_05901-06000.json
db_2016_06001-06100.json
db_2016_06101-06200.json
db_2016_06201-06300.json
db_2016_06301-06400.json
db_2016_06401-06500.json
db_2016_06501-06600.json
db_2016_06601-06700.json
db_2016_06701-06800.json
db_2016_06801-06900.json
db_2016_06901-07000.json
db_2016_07001-07100.json
db_2016_07101-07200.json
db_2016_07201-07300.json
db_2016_07301-07400.json
db_2016_07401-07500.json
db_2017_00001-00100.json
db_2017_00101-00200.json
db_2017_00201-00300.json
db_2017_00301-00400.json
db_2017_00401-00500.json
db_2017_00501-00600.json
db_2017_00601-00700.json
db_2017_00701-00800.json
db_2017_00801-00900.json
db_2017_00901-01000.json
db_2017_01001-01100.json
db_2017_01101-01200.json
db_2017_01201-01300.json
db_2017_01301-01400.json
db_2017_01401-01500.json
db_2017_01501-01600.json
db_2017_01601-01700.json
db_2017_01701-01800.json


db_2020_09301-09400.json
db_2020_09401-09500.json
db_2021_00001-00100.json
db_2021_00101-00200.json
db_2021_00201-00300.json
db_2021_00301-00400.json
db_2021_00401-00500.json
db_2021_00501-00600.json
db_2021_00601-00700.json
db_2021_00701-00800.json
db_2021_00801-00900.json
db_2021_00901-01000.json
db_2021_01001-01100.json
db_2021_01101-01200.json
db_2021_01201-01300.json
db_2021_01301-01400.json
db_2021_01401-01500.json
db_2021_01501-01600.json
db_2021_01601-01700.json
db_2021_01701-01800.json
db_2021_01801-01900.json
db_2021_01901-02000.json
db_2021_02001-02100.json
db_2021_02101-02200.json
db_2021_02201-02300.json
db_2021_02301-02400.json
db_2021_02401-02500.json
db_2021_02501-02600.json
db_2021_02601-02700.json
db_2021_02701-02800.json
db_2021_02801-02900.json
db_2021_02901-03000.json
db_2021_03001-03100.json
db_2021_03101-03200.json
db_2021_03201-03300.json
db_2021_03301-03400.json
db_2021_03401-03500.json
db_2021_03501-03600.json
db_2021_03601-03700.json
db_2021_03701-03800.json


In [74]:
#chart excel 파일 다운로드
dowload_chart_excelfile_by_api(today, chart_excelfile_path)

저장되었습니다.


In [88]:
#chart excel 파일 csv로 바꾸기
df = load_chart(today, chart_excelfile_path)
df = chart_data_update(df)
df = chart_column_update(df)
df.to_csv(chart_db_csvfile_path+'\\chart_'+today+'.csv', mode='w', encoding='utf8', index=False)

In [12]:
#엑셀파일 다운로드
def download_excelfile_by_api(today, excelfile_path):
    excelfile_url='https://nedrug.mfds.go.kr/cmn/xls/down/OpenData_ItemPermitDetail'
    excelfile_download_path = excelfile_path + '\\OpenData_ItemPermitDetail'+today+'.xls'

    request.urlretrieve(excelfile_url, excelfile_download_path)
    print("저장되었습니다.")

#엑셀파일 내용 읽기
def load_file(date, path):
    excelfile_load_path = path +'\\OpenData_ItemPermitDetail'+date+'.xls'
    df = pd.read_excel(excelfile_load_path)
    return df

#저장 또는 업데이트 할 내용의 리스트를 반환한다.
def get_list_from_db(save_or_update, df, start_date = '', end_date = '', type = ''):
    if(save_or_update=='save'):
        df1 = df.loc[(df['품목일련번호'] >= 195500000) & (df['품목일련번호'] < (int(today[0:4])+1)*100000)]
    elif(save_or_update=='update'):
        st_day_int = int(start_date.strftime('%Y%m%d'))
        end_day_int = int(end_date.strftime('%Y%m%d'))
        df1 = df.loc[(df[type] >= st_day_int) & (df[type] <= end_day_int)]
    df1 = df1.fillna('')
    print('get_list_from_db end & make_db_json_list start')
    time_record_make_start = datetime.now(KST)
    print(time_record_make_start)
    out_json = make_db_json_list(df1)
    print('make_db_json_list end & get_list_from_db end')
    time_record_make_end = datetime.now(KST)
    print(time_record_make_end)
    return out_json

#{"195500002": {"item_seq": "195500002", "item_name": "종근당염산에페드린정", "entp_name": "(주)종근당", "chart": "본품은 백색의 정제다.", "item_permit_date": "19550117", "edi_code": "", "permit_kind_name": "허가", "cancel_name": "유효기간만료", "cancel_date": "20200101", "etc_otc_code": "전문의약품", "main_item_ingr": "[M040420]염산에페드린", "atc": "R03CA02"}, ... }
def make_db_json_list(df):
    out_json = {}
    check_file_name = []
    for idx in df.index : 
        file_name = get_file_name(str(df['품목일련번호'][idx]))
        item_seq = str(df['품목일련번호'][idx])
        data = get_data_dbjson(idx, df)
        out_json = push_into_out_json(file_name, out_json, item_seq, data)
    return out_json

#dbjson에 넣을 항목에 따라 저장한다
def get_data_dbjson(idx, df):
    data = OrderedDict()  
    for key, value in def_db.items():
        data[key] = str(df[value][idx])
        if(value=='취소일자' or value=='변경일자'):
            data[key] = str(df[value][idx])[0:8]
        if(value=='표준코드'):
            data[key] = str(df[value][idx])[0:13]
    return data

#195500002 -> 1955_00001-00100
def get_file_name(item_seq):
    year = int(item_seq[0:4])
    seq = int(item_seq[4:9])
    temp = int((seq-1)/100)
    range_st = str(temp*100+1)
    range_end = str(temp*100+100)
    five_zeros = '00000'
    range_st = five_zeros[:5-len(range_st)]+range_st
    range_end = five_zeros[:5-len(range_end)]+range_end
    return str(year)+'_'+range_st+'-'+range_end

#파일 저장 또는 업데이트 시작
def save_or_update(out_json, type, file_path):
    file_list = os.listdir(file_path)
    for file_name in out_json:
        print(file_name)
        full_file_name = type+'_'+file_name+".json"
        if file_name in file_list:
            with open(file_path+'/'+full_file_name, 'r', encoding='utf8') as file:
                content = file.read()
            content_json = json.loads(content)
            for item_seq in out_json[file_name]:
                content_json[item_seq] = out_json[file_name][item_seq]
            obj = json.dumps(content_json, ensure_ascii=False)
            time.sleep(0.1)
        else:
            content_json = {}
            for item_seq in out_json[file_name]:
                content_json[item_seq] = out_json[file_name][item_seq]
            obj = json.dumps(content_json, ensure_ascii=False)
            time.sleep(0.1)
            file_list.append(full_file_name)
        with open(file_path+'/'+full_file_name, 'w', encoding='utf8') as file:
            file.write(obj)
            time.sleep(0.1)

#document 파일로부터 out_json 생성
def get_list_from_document(folder_path):
    xmlfile_folder_list = os.listdir(folder_path)
    out_json={}
    for item_seq in xmlfile_folder_list:
        print(item_seq)
        data = {}
        ee = get_content_from_document(folder_path+'/'+item_seq+'/EE_DOC_DATA.xml')
        ud = get_content_from_document(folder_path+'/'+item_seq+'/UD_DOC_DATA.xml')
        nb = get_content_from_document(folder_path+'/'+item_seq+'/NB_DOC_DATA.xml')
        data['EE_DOC_DATA'] = xml_to_json(ee)
        data['UD_DOC_DATA'] = xml_to_json(ud)
        data['NB_DOC_DATA'] = xml_to_json(nb)
        file_name = get_file_name(item_seq)
        out_json = push_into_out_json(file_name, out_json, item_seq, data)
        #print(data)
    return out_json

#document의 xml파일을 읽어 beautifulsoup 형태로 반환
def get_content_from_document(file_path):
    if not os.path.isfile(file_path):
        data = {}
        return data
    with open(file_path, 'r', encoding='utf8') as file:
        contents = file.read()
        soup = BeautifulSoup(contents, 'xml')
    return soup

#api로부터 누락된 시퀀스의 out_json 생성
def get_list_from_api(missing_seq_arr):
    serviceKey = 'idexeeezUsEPdbnJ%2BWNFg1ImgUZ21EIA%2BzhJbHrUn3NA%2FoAzR3YTaTPH2nTXKGuSA%2BxjyemKa81puLL303Yiww%3D%3D'
    out_json = {}
    for item_seq in missing_seq_arr:
        url = 'http://apis.data.go.kr/1471000/DrugPrdtPrmsnInfoService02/getDrugPrdtPrmsnDtlInq01?serviceKey='+serviceKey+'&item_seq='+item_seq
        print(item_seq)
        response = requests.get(url)
        contents = response.text
        soup = BeautifulSoup(contents, 'xml')
        data = {}
        ee = soup.find('EE_DOC_DATA')
        ud = soup.find('UD_DOC_DATA')
        nb = soup.find('NB_DOC_DATA')
        data['EE_DOC_DATA'] = xml_to_json(ee)
        data['UD_DOC_DATA'] = xml_to_json(ud)
        data['NB_DOC_DATA'] = xml_to_json(nb)
        file_name = get_file_name(item_seq)
        out_json = push_into_out_json(file_name, out_json, item_seq, data)
    return out_json

#out_json에서 item_seq의 리스트를 반환한다.
def get_item_seq_list_from_out_json(out_json):
    item_seq_list = []
    for file_name in out_json:
        for item_seq in out_json[file_name]:
            item_seq_list.append(item_seq)
    return item_seq_list

#out_json에 데이터 집어넣기
def push_into_out_json(file_name, out_json, item_seq, data):
    if file_name in out_json:
        out_json[file_name][item_seq] = data
    else:
        out_json[file_name] = {}
        out_json[file_name][item_seq] = data
    return out_json

#beautifulsoup의 xml을 json 형태로 바꾸기
def xml_to_json(content):
    data = []
    if content=={} or content is None:
        return data
    soup=content
    for docElement in soup.find_all('DOC'):
        dataDoc = {}
        dataDoc['title'] = docElement['title']
        dataDoc['sections'] = []
        for sectionElement in soup.find_all('SECTION'):
            dataSection = {}
            dataSection['title'] = sectionElement['title']
            dataSection['articles'] = []
            for articleElement in soup.find_all('ARTICLE'):
                dataArticle = {}
                dataArticle['title'] = articleElement['title']
                dataArticle['paragraphs'] = []
                for paragraphElement in soup.find_all('PARAGRAPH'):
                    if type(paragraphElement.string) is type(None):
                        continue
                    dataParagraph = {}
                    dataParagraph['tag'] = paragraphElement['tagName']
                    dataParagraph['content'] = paragraphElement.string
                    dataArticle['paragraphs'].append(dataParagraph)
                dataSection['articles'].append(dataArticle)
            dataDoc['sections'].append(dataSection)
        data.append(dataDoc)
    return data

#document파일의 정보 중 빠진 item_seq 리스트를 반환한다.
def get_missing_seq(df, documentfile_path):
    missing_seq_list = []
    cur_document_seq_list = []
    item_seq_list = df['품목일련번호'].to_list()
    item_seq_list = list(map(str, item_seq_list))
    documentfile_name_list = os.listdir(documentfile_path)
    for file_name in documentfile_name_list:
        print(file_name)
        with open(documentfile_path+'\\'+ file_name, 'r', encoding='utf8') as file:
            content = file.read()
        content_json = json.loads(content)
        for item_seq in content_json:
            cur_document_seq_list.append(item_seq)
    for item_seq in item_seq_list:
        if not item_seq in cur_document_seq_list:
            missing_seq_list.append(item_seq)
    return missing_seq_list

#json파일을 dataframe object로 바꿔서 반환한다
def json_to_pd(file_path):
    pdObj = pd.read_json(file_path, orient='index', encoding='utf8')
    return pdObj

#컬럼명 리스트, 데이터 리스트를 파라미터로 입력받으면 csv파일을 생성한다
def write_csv(header, data, check):
    with open(csvfile_path+'\\db.csv', 'a', encoding='utf8', newline='') as file:
        writer = csv.writer(file)
        if(check == True):
            check = False
            # write the header
            writer.writerow(header)
        # write multiple rows
        writer.writerows(data)  

#features 엑셀 파일 다운로드
def dowload_chart_excelfile_by_api(today, chart_excelfile_path):
    chart_excelfile_url = "https://nedrug.mfds.go.kr/cmn/xls/down/OpenData_PotOpenTabletIdntfc"
    chart_excelfile_download_path = chart_excelfile_path + '\\chart_'+today+'.xls'
    request.urlretrieve(chart_excelfile_url, chart_excelfile_download_path)
    print("저장되었습니다.")

#features 엑셀 파일을 dataframe으로 반환
def load_chart(today, path):
    excelfile_load_path = path +'\\chart_'+today+'.xls'
    df = pd.read_excel(excelfile_load_path)
    df = df.fillna('')
    return df

#chart 데이터 가공
def chart_data_update(df):
    change_date_list = df['변경일자'].to_list()
    class_no_list = df['분류번호'].to_list()
    change_date_list = list(map(str, change_date_list))
    class_no_list = list(map(str, class_no_list))
    for idx in range(len(change_date_list)):
        change_date_list[idx] = change_date_list[idx][0:8]
        class_no_list[idx] = change_date_list[idx][0:5]
    df['변경일자'] = pd.Series(change_date_list)
    df['분류번호'] = pd.Series(class_no_list)
    return df
#features의 컬럼 값을 영어로 바꿔줌
def chart_column_update(df):
    reversed_def_chart = {v:k for k,v in def_chart.items()}
    column_list = []
    for column in df.columns:
        column_list.append(reversed_def_chart[column])
    df.columns = column_list
    return df

In [7]:
zip_file_path = 'C:\\Users\\user\\Medicine\\zipfiles\\2016-2020.zip'
extract_path = 'C:\\Users\\user\\Medicine\\unzip\\2016-2020'
  
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)